In [1]:

from goatools.test_data.genes_NCBI_10090_ProteinCoding import GENEID2NT as GeneID2nt_mouse

from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

"""
This chunk initializes necessary variables for GO analysis below 
"""
obo_fname = download_go_basic_obo()
fin_gene2go = download_ncbi_associations()
obodag = GODag("go-basic.obo")
#run one time to initialize
mapper = {}

for key in GeneID2nt_mouse:
    mapper[GeneID2nt_mouse[key].Symbol] = GeneID2nt_mouse[key].GeneID
    
inv_map = {v: k for k, v in mapper.items()}
#run one time to initialize

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[10090])
# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()
#run one time to initialize
goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_mouse.keys(), # List of mouse protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method
#run one time to initialize
GO_items = []

temp = goeaobj.ns2objgoea['BP'].assoc
for item in temp:
    GO_items += temp[item]
    

temp = goeaobj.ns2objgoea['CC'].assoc
for item in temp:
    GO_items += temp[item]
    

temp = goeaobj.ns2objgoea['MF'].assoc
for item in temp:
    GO_items += temp[item]
#pass list of gene symbols
def go_it(test_genes):
    print(f'input genes: {len(test_genes)}')
    
    mapped_genes = []
    for gene in test_genes:
        try:
            mapped_genes.append(mapper[gene])
        except:
            pass
    print(f'mapped genes: {len(mapped_genes)}')
    
    goea_results_all = goeaobj.run_study(mapped_genes)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, x.p_uncorrected, x.p_fdr_bh,\
                   x.ratio_in_study[0], x.ratio_in_study[1], GO_items.count(x.GO), list(map(lambda y: inv_map[y], x.study_items)),\
                   ], goea_results_sig)), columns = ['GO', 'term', 'class', 'p', 'p_corr', 'n_genes',\
                                                    'n_study', 'n_go', 'study_genes'])

    GO = GO[GO.n_genes > 1]
    return GO

  EXISTS: go-basic.obo
  EXISTS: gene2go
go-basic.obo: fmt(1.2) rel(2023-01-01) 46,739 Terms
HMS:0:00:03.971830 429,754 annotations, 29,773 genes, 19,208 GOs, 1 taxids READ: gene2go 

Load BP Ontology Enrichment Analysis ...
 63% 17,638 of 28,212 population items found in association

Load CC Ontology Enrichment Analysis ...
 66% 18,670 of 28,212 population items found in association

Load MF Ontology Enrichment Analysis ...
 60% 17,045 of 28,212 population items found in association


In [3]:
import pandas as pd 
import numpy as np 
import os 
import sys 

In [4]:
GeneSets = pd.read_excel('../output/GO_analysis/GeneSet_Comparison.xlsx').iloc[:,1:]

In [5]:
"""
Conduct GO analysis on all the columns in GeneSets from different anlaysis
into the df dictionary 
"""
df = {}
for col in GeneSets.columns: 
    df[col] = go_it(GeneSets[col].values)
    df[col]['n_genes/n_go'] = df[col].n_genes/df[col].n_go
    df[col]['n_genes/n_study'] = df[col].n_genes/df[col].n_study

input genes: 99
mapped genes: 96

Runing BP Ontology Analysis: current study set of 96 IDs.
 90%     86 of     96 study items found in association
100%     96 of     96 study items found in population(28212)
Calculating 12,766 uncorrected p-values using fisher_scipy_stats
  12,766 terms are associated with 17,638 of 28,212 population items
     718 terms are associated with     86 of     96 study items
  METHOD fdr_bh:
       1 GO terms found significant (< 0.05=alpha) (  1 enriched +   0 purified): statsmodels fdr_bh
       3 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Runing CC Ontology Analysis: current study set of 96 IDs.
 95%     91 of     96 study items found in association
100%     96 of     96 study items found in population(28212)
Calculating 1,804 uncorrected p-values using fisher_scipy_stats
   1,804 terms are associated with 18,670 of 28,212 population items
     208 terms are associated with

   1,804 terms are associated with 18,670 of 28,212 population items
      94 terms are associated with     49 of     50 study items
  METHOD fdr_bh:
       3 GO terms found significant (< 0.05=alpha) (  3 enriched +   0 purified): statsmodels fdr_bh
       5 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Runing MF Ontology Analysis: current study set of 50 IDs.
 84%     42 of     50 study items found in association
100%     50 of     50 study items found in population(28212)
Calculating 4,585 uncorrected p-values using fisher_scipy_stats
   4,585 terms are associated with 17,045 of 28,212 population items
     117 terms are associated with     42 of     50 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (p

In [47]:

import plotly.express as px

for i in df.keys():
    fig = px.bar(df[i], x='n_genes/n_go', y='term', orientation='h',
                hover_data=['study_genes'],
                title = i).update_layout(
#                                 template='plotly_dark',
                                plot_bgcolor='rgba(0, 0, 0, 0)',
#                                 paper_bgcolor='rgba(0, 0, 0, 0)',
                            )
    fig.show()
    fig.write_html("../output/GO_analysis/GO_"+i+"_.html")

In [49]:

import plotly.express as px

for i in df.keys():
    fig = px.bar(df[i], x='n_genes', y='term', orientation='h',
                hover_data=['study_genes'],
                title = i).update_layout(
#                                 template='plotly_dark',
                                plot_bgcolor='rgba(0, 0, 0, 0)',
#                                 paper_bgcolor='rgba(0, 0, 0, 0)',
                            )
    fig.show()
    fig.write_html("../output/GO_analysis/GO_ngenes_"+i+"_.html")

In [111]:
import GE_functions

In [116]:
"""
PCA based on the 'term' from GO analysis 
"""

In [271]:
for i in df.keys():
    df[i]['data'] = i 
new_df = pd.concat([df['mse_rank_log1p_ALL_Olfr_rtp1'],
           df['mse_rank_log1p_normExp_ALL_Olfr_rtp1'], 
           df['mse_rank_normalized_ALL_Olfr_rtp1'], 
           df['mse_rank_normalized_normExp_ALL_Olfr_rtp1'], 
           df['human_cov_log2FC_rank'], 
           df['human_cov_padj_rank'], 
           df['bkRNA_Rank'], 
           df['msViper_Rank']])

In [272]:
temp = new_df[['term','n_genes', 'data']].groupby(['term', 'data'], as_index=False).sum()
temp = temp.pivot(index='term', columns='data', values='n_genes').fillna(0).transpose()
temp = temp.drop('msViper_Rank')

In [273]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(temp) 

variance_ratio = pca.explained_variance_ratio_
pc_label = []
for i, var in enumerate(variance_ratio):
    pc_label.append("PC{}_({}%)".format(i, round(var*100, 2)))
    
pca_df = pd.DataFrame(
    data=principal_components,
    columns=pc_label,
    index=temp.index
)
# Labels where dataset is from 
pca_df.loc[pca_df.index.str.contains('mse_rank'), 'from']  = 'Brann'
pca_df.loc[pca_df.index.str.contains('msViper|bkRNA'), 'from'] = 'Shayya'
pca_df.loc[pca_df.index.str.contains('human_cov'), 'from']  = 'Zazhytska'

In [274]:
fig = px.scatter(pca_df, x=pca_df.columns[0], 
                         y=pca_df.columns[1], 
                 color='from', 
                title = "")
fig.update_traces(marker=dict(size=15))
fig.show()
fig.write_html("../output/GO_analysis/PCA_GO_2.html")